In [ ]:
%load_ext autoreload

%autoreload 2
import gcsfs
import numpy as np
import zarr
fs = gcsfs.GCSFileSystem()
import torch


In [ ]:
from oxeo.water.models.utils import load_tile, TilePath, tile_from_id
from oxeo.water.models.segmentation import Segmentation2DPredictor

tile_path = TilePath(tile_from_id("43_P_10000_65_137"), "landsat-8")

seg_predictor = Segmentation2DPredictor(
    batch_size=16,
    ckpt_path="gs://oxeo-models/semseg/epoch_012.ckpt",
    input_channels=6,
    num_classes=3,
    chip_size=250,
    fs=fs,
)


In [ ]:
res = seg_predictor.predict(
    fs.get_mapper,
    tile_path,
    revisit=slice(0, 2),
    bands=["nir", "red", "green", "blue", "swir1", "swir2"],
    target_size=1000
)


In [ ]:
tile = load_tile(
            fs_mapper=fs.get_mapper,
            tile_path=tile_path,
            masks=("pekel", "cloud_mask"),
            revisit=slice(0, 2),
    bands=["nir", "red", "green", "blue", "swir1", "swir2"],
        )

In [ ]:
import matplotlib.pyplot as plt
from skimage.exposure import rescale_intensity
import os
os.environ["LOGURU_LEVEL"] = "INFO"
i = 1
plt.imshow(res[i])
plt.show()

plt.imshow(tile["pekel"].numpy().squeeze()[i])
plt.show()


img = tile["image"].numpy()[i][[1, 2, 3]].transpose(1, 2, 0)
vmin, vmax = np.percentile(img, q=(2, 98))
img = rescale_intensity(img, in_range=(vmin, vmax), out_range=(0, 1))
plt.imshow(img)
